## 이벤트 처리

In [ ]:
# p.52 키 이벤트
import cv2

img_file = './img/girl.jpg'
img = cv2.imread(img_file)
title = 'IMG'
x, y = 100, 100

while True:
    cv2.imshow(title, img)
    cv2.moveWindow(title, x, y)
    key = cv2.waitKey(0) & 0xFF
    print(key, chr(key))
    if key == ord('h'):
        x -= 10
    elif key == ord('j'):
        y += 10
    elif key == ord('k'):
        y -= 10
    elif key == ord('l'):
        x += 10
    elif key == ord('q') or key == 27: # 'q' 또는 'esc' 키이면 종료
        break
        cv2.destroyAllWindows()
    cv2.moveWindow(title, x, y)

In [ ]:
# p.53 mouse event
import cv2

title = 'mouse event'
img = cv2.imread('./img/blank_500.jpg')
cv2.imshow(title, img)

def onMouse(event, x, y, flags, param):
    print(event, x, y, )
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(img, (x, y), 30, (0, 0, 0), -1)
        cv2.imshow(title, img)

cv2.setMouseCallback(title, onMouse)

while True:
    if cv2.waitKey(0) & 0xFF ==27:  # esc로 종료
        break
cv2.destroyAllWindows()

In [11]:
# p.56 flag를 이용한 color circle
import cv2

title = 'mouse event'
img = cv2.imread('./img/blank_500.jpg')
cv2.imshow(title, img)

colors = {
    'black': (0, 0, 0),
    'red': (0, 0, 255),
    'blue': (255, 0, 0),
    'green': (0, 255, 0)
}

def onMouse(event, x, y, flags, param):
    print(event, x, y, flags)
    color = colors['black']
    if event == cv2.EVENT_LBUTTONDOWN:
        if flags & cv2.EVENT_FLAG_CTRLKEY and flags & cv2.EVENT_FLAG_SHIFTKEY:
            color = colors['green']
        elif flags & cv2.EVENT_FLAG_SHIFTKEY:
            color = colors['blue']
        elif flags & cv2.EVENT_FLAG_CTRLKEY:
            color = colors['red']

        cv2.circle(img, (x, y), 30, color, -1)
        cv2.imshow(title, img)
cv2.setMouseCallback(title, onMouse)

while True:
    if cv2.waitKey(0) & 0xFF == 27: # esc 버튼으로 종료
        break
    
cv2.destroyAllWindows()

0 455 241 0
0 423 241 0
0 396 241 0
0 357 245 0
0 341 246 0
0 335 246 0
0 330 246 0
0 330 248 0
0 329 248 0
0 327 246 0
0 324 245 0
0 323 245 0
0 321 245 0
0 320 245 0
0 318 244 0
0 315 244 0
0 315 242 0
0 314 242 0
1 314 242 1
0 314 241 1
4 314 241 0
0 321 232 0
0 321 232 0
0 324 226 0
0 330 215 0
0 333 211 0
0 335 202 0
0 336 200 0
0 336 199 0
0 335 199 0
0 333 197 0
0 332 196 0
0 330 194 0
0 330 193 0
0 329 193 0
0 329 191 0
0 327 191 0
1 327 191 1
4 327 191 0
0 327 191 0
0 326 191 0
0 326 190 0
0 324 190 0
0 323 190 0
0 323 188 0
0 321 185 0
0 318 182 0
0 318 181 0
0 317 178 0
0 315 176 0
0 315 175 0
0 314 175 0
0 314 173 0
0 312 173 0
0 312 172 0
0 311 172 0
0 309 170 0
0 311 170 0
0 312 172 0
0 314 173 0
0 315 173 0
0 315 175 0
0 317 175 0
0 317 176 0
0 318 176 0
1 318 176 1
4 318 176 0
0 318 176 0
0 317 179 0
0 315 182 0
0 312 185 0
0 311 187 0
0 309 188 0
0 306 191 0
0 306 193 0
1 306 193 1
4 306 193 0
0 306 193 0
0 305 196 0
0 303 199 0
0 302 200 0
0 302 203 0
0 300 205 0
0 30

In [12]:
import cv2
import numpy as np

win_name = 'Trackbar'

img = cv2.imread('./img/blank_500.jpg')
cv2.imshow(win_name, img)

# 트랙바 이벤트 처리를 위한 함수
def onChange(x):
    print(x)
    r = cv2.getTrackbarPos('R', win_name)
    g = cv2.getTrackbarPos('G', win_name)
    b = cv2.getTrackbarPos('B', win_name)
    print(r, g, b)
    img[:] = [b, g, r]
    cv2.imshow(win_name, img)

cv2.createTrackbar('R', win_name, 255, 255, onChange)
cv2.createTrackbar('G', win_name, 255, 255, onChange)
cv2.createTrackbar('B', win_name, 255, 255, onChange)

while True:
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()

255


error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-1i5nllza\opencv\modules\highgui\src\window.cpp:868: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


255


error: OpenCV(4.5.3) C:\Users\runneradmin\AppData\Local\Temp\pip-req-build-1i5nllza\opencv\modules\highgui\src\window.cpp:868: error: (-215:Assertion failed) trackbar in function 'cv::getTrackbarPos'


255
255 255 255
254
254 255 255
251
251 255 255
248
248 255 255
244
244 255 255
241
241 255 255
233
233 255 255
230
230 255 255
225
225 255 255
221
221 255 255
219
219 255 255
216
216 255 255
214
214 255 255
213
213 255 255
212
212 255 255
210
210 255 255
208
208 255 255
207
207 255 255
206
206 255 255
205
205 255 255
203
203 255 255
198
198 255 255
195
195 255 255
190
190 255 255
188
188 255 255
187
187 255 255
184
184 255 255
181
181 255 255
178
178 255 255
175
175 255 255
172
172 255 255
168
168 255 255
165
165 255 255
159
159 255 255
157
157 255 255
155
155 255 255
154
154 255 255
149
149 255 255
147
147 255 255
143
143 255 255
137
137 255 255
135
135 255 255
134
134 255 255
132
132 255 255
130
130 255 255
128
128 255 255
127
127 255 255
126
126 255 255
125
125 255 255
123
123 255 255
120
120 255 255
118
118 255 255
117
117 255 255
116
116 255 255
112
112 255 255
110
110 255 255
109
109 255 255
106
106 255 255
103
103 255 255
100
100 255 255
99
99 255 255
94
94 255 255
90
90 255 25

KeyboardInterrupt: 